### **Combining AWS Lambda with API Gateway to Handle HTTP Requests**

To integrate AWS Lambda with API Gateway, you can use either the AWS Management Console (interface) or the AWS CLI (terminal). This integration allows you to expose your Lambda function as a REST or HTTP API endpoint.

---

### **1. Using the AWS Management Console (Interface)**

#### **Step 1: Create a Lambda Function**
1. Follow the steps [here](#2-write-your-python-lambda-function) to create a Lambda function (e.g., `my-python-function`).

---

#### **Step 2: Create an API in API Gateway**
1. Go to the [API Gateway Console](https://console.aws.amazon.com/apigateway/).
2. Click **Create API**.
3. Choose one of the following:
   - **HTTP API** (recommended for lower latency and cost).
   - **REST API** (offers more features).
4. Select **Build**.

---

#### **Step 3: Configure the API**
1. **For HTTP API**:
   - Click **Add Integration** → Select **Lambda function**.
   - Search for and select your Lambda function (e.g., `my-python-function`).
   - Click **Add Integration**.
   - Add a route:
     - Method: `ANY` or specific HTTP method (e.g., `POST`).
     - Path: `/example`.
     - Integration target: Select your Lambda function.
   - Deploy your API by clicking **Deploy** → Create a new stage (e.g., `prod`).
   - Note the **Invoke URL** for testing.

2. **For REST API**:
   - Create a new resource (e.g., `/example`).
   - Create a new method (e.g., `POST`).
   - Choose **Integration type** → Select **Lambda Function**.
   - Enter your Lambda function name.
   - Deploy your API:
     - Click **Actions** → **Deploy API**.
     - Create a new stage (e.g., `prod`).
   - Note the **Invoke URL**.

---

#### **Step 4: Test Your API**
1. Use a tool like Postman, Curl, or your browser to send a request to the API endpoint.
2. Example (assuming HTTP API and a `POST` request):
   ```bash
   curl -X POST -H "Content-Type: application/json" -d '{"key":"value"}' https://{api-id}.execute-api.{region}.amazonaws.com/prod/example
   ```

---

### **2. Using AWS CLI (Terminal)**

#### **Step 1: Create a Lambda Function**
Follow the steps [here](#5-create-a-lambda-function-via-aws-cli) to create a Lambda function.

---

#### **Step 2: Create an API Gateway HTTP API**
1. Create a new HTTP API:
   ```bash
   aws apigatewayv2 create-api \
       --name "MyHttpApi" \
       --protocol-type HTTP \
       --target arn:aws:lambda:{region}:{account-id}:function:my-python-function
   ```

2. Note the `ApiId` from the response.

---

#### **Step 3: Configure a Route and Stage**
1. Add a route to your API:
   ```bash
   aws apigatewayv2 create-route \
       --api-id {ApiId} \
       --route-key "POST /example"
   ```

2. Add an integration to your Lambda function:
   ```bash
   aws apigatewayv2 create-integration \
       --api-id {ApiId} \
       --integration-type AWS_PROXY \
       --integration-uri arn:aws:lambda:{region}:{account-id}:function:my-python-function
   ```

3. Deploy the API:
   ```bash
   aws apigatewayv2 create-deployment \
       --api-id {ApiId} \
       --stage-name prod
   ```

---

#### **Step 4: Grant API Gateway Permissions to Lambda**
API Gateway needs permission to invoke your Lambda function:
```bash
aws lambda add-permission \
    --function-name my-python-function \
    --statement-id apigateway-invoke \
    --action lambda:InvokeFunction \
    --principal apigateway.amazonaws.com \
    --source-arn arn:aws:execute-api:{region}:{account-id}:{ApiId}/*
```

---

#### **Step 5: Test Your API**
1. Note the invoke URL for your API:
   ```bash
   aws apigatewayv2 get-api --api-id {ApiId}
   ```

2. Test using Curl or Postman:
   ```bash
   curl -X POST -H "Content-Type: application/json" -d '{"key":"value"}' https://{ApiId}.execute-api.{region}.amazonaws.com/prod/example
   ```

---

### **3. Notes and Tips**
- **Error Debugging**:
  - Check logs in **CloudWatch** for both Lambda and API Gateway.
- **IAM Permissions**:
  - Ensure the Lambda execution role and API Gateway have the required permissions.
- **Custom Domains**:
  - Attach a custom domain to your API Gateway if desired.

Let me know if you'd like detailed guidance for any step!